# Opening a YouTube Link

There are two methods: 
1. Use package `webbrowser` to open the link in the default web-browser.
1. Use package `selenium` to open a browser (here we will use Mozilla-Firefox).

In [7]:
import webbrowser as wb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, sys, glob
import re, time
from IPython.display import display, clear_output, YouTubeVideo
from shutil import copyfile # copyfile(src, dst)

import errno
import requests
from bs4 import BeautifulSoup as bsp
import urllib.parse as urlparse

from tqdm import tqdm, tnrange, tqdm_notebook

%matplotlib inline
%config InlineBackend.figure_format = 'svg' # 'svg', 'retina'
plt.style.use('seaborn-white')

# Open Link with python's `webbrowser` package

In [22]:
target_url = "https://www.youtube.com/watch?v=RJ8PPwv-tEE"
tab = wb.open_new_tab(target_url)

In [41]:
wb.open(target_url, new=1)

True

In [42]:
wb.open(target_url, new=1, autoraise=True)
time.sleep(1.5)
for i in range(10):
    wb.open_new_tab(target_url)
    time.sleep(1.5)

# Open Link with `selenium` package

### Note: Download and Point To the Path of the Browser Driver

+ For first time use download GECKODRIVER from [here](https://github.com/mozilla/geckodriver/releases)
+ Follow the instructions given [here]() to point to the downloaded driver.

```
from selenium import webdriver
driver_location = r'your\path\geckodriver.exe'
target_url = 'http://inventwithpython.com'
driver = webdriver.Firefox(executable_path = driver_location)
driver.get(target_url)

```

## Import Packages and Define Custom Function

In [60]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def getWebDriver(browser_name="firfox", target_url=""):
    
    default_browser_name = "firefox"
    supported_browsers_list = ["firefox","chrome"]
    
    if browser_name not in supported_browsers_list:
        browser_name = default_browser_name
    
    if (browser_name == "firefox"):
        driver_path = r'C:\Users\raysu\Anaconda3\selenium\webdriver\firefox'
        driver_name = "geckodriver.exe"
    elif (browser_name == "chrome"):
        driver_path = r'C:\Users\raysu\Anaconda3\selenium\webdriver\chrome'
        driver_name = "chromedriver.exe"
    
    driver_executable_path = os.path.join(driver_path,driver_name)
    
    print(driver_executable_path)
    if (browser_name == "firefox"):
        driver = webdriver.Firefox(executable_path = driver_executable_path)
    if (browser_name == "chrome"):
        driver = webdriver.Chrome(executable_path = driver_executable_path)
    
    if not (target_url.strip() == ""):
        driver.get(target_url)
    
    return driver
    

## Open New Tab and Load URL

The most prescribed method for this is using ActionChains: 
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('t').key_up(Keys.CONTROL).perform()
```
However, this did not work for me. Instead, the following method ([source](https://gist.github.com/lrhache/7686903)) worked in opening a new tab:  
```python
driver.execute_script('''window.open("about:blank", "_blank");''')
```
**Refernces**:  
+ https://stackoverflow.com/questions/10629815/how-to-switch-to-new-window-in-selenium-for-python
+ https://stackoverflow.com/questions/5859119/has-anyone-used-actionchains-of-webdriverpython-binding
+ https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.action_chains
+ https://seleniumhq.github.io/selenium/docs/api/py/webdriver/selenium.webdriver.common.action_chains.html



## Complete code block:  

```python
# import statements
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

from tqdm import tqdm, tnrange, tqdm_notebook
import time, os

# define custom browser creation function
def getWebDriver(browser_name="firfox", target_url=""):
    
    default_browser_name = "firefox"
    supported_browsers_list = ["firefox","chrome"]
    
    if browser_name not in supported_browsers_list:
        browser_name = default_browser_name
    
    if (browser_name == "firefox"):
        driver_path = r'C:\Users\raysu\Anaconda3\selenium\webdriver\firefox'
        driver_name = "geckodriver.exe"
    elif (browser_name == "chrome"):
        driver_path = r'C:\Users\raysu\Anaconda3\selenium\webdriver\chrome'
        driver_name = "chromedriver.exe"
    
    driver_executable_path = os.path.join(driver_path,driver_name)
    
    print(driver_executable_path)
    if (browser_name == "firefox"):
        driver = webdriver.Firefox(executable_path = driver_executable_path)
    if (browser_name == "chrome"):
        driver = webdriver.Chrome(executable_path = driver_executable_path)
    
    if not (target_url.strip() == ""):
        driver.get(target_url)
    
    return driver
    
    
# define target_url and create webdriver for firefox browser
target_url = r"https://www.youtube.com/watch?v=RJ8PPwv-tEE"
driver = getWebDriver('firefox', target_url='')

# Switch to first window_handle, open target_url and 
#    then recursively: 
#    1. open new tabs, 
#    2. switch to that tab's window_handle, 
#    3. open target_url
driver.switch_to.window(driver.window_handles[0])
driver.get(target_url)
for i in tqdm_notebook(range(1,10), desc='Mozilla Loop'):
    # open new tab
    driver.execute_script('''window.open("about:blank", "_blank");''')
    # switch to new tab
    driver.switch_to.window(driver.window_handles[i])
    driver.get(target_url)
    
# close all the open tabs (one by one)
print('Open Tabs: {}'.format(len(driver.window_handles)))
for i, window_handle in tqdm_notebook(enumerate(driver.window_handles), desc='Close Tabs'):
    driver.switch_to.window(window_handle)
    driver.close()
    try:
        open_tabs_left = len(driver.window_handles)
    except:
        open_tabs_left = 0
    print('{}. Open Tabs: {} | Closed: {}'.format(i, open_tabs_left, window_handle))
    time.sleep(0.5)
```

## Create a Browser Driver

In [20]:
target_url = r"https://www.youtube.com/watch?v=RJ8PPwv-tEE"
driver = getWebDriver('firefox', target_url='')

C:\Users\raysu\Anaconda3\selenium\webdriver\firefox\geckodriver.exe


Print the window handles. If there are multiple windows, or tabs open, this would return multiple window handles (as a list of strings).

In [78]:
window_handles = driver.window_handles
print(window_handles)

['6442450945']


## Implementation

### Open multiple browser tabs and then load URL in each of them

In [85]:
# define target_url and create webdriver for firefox browser
target_url = r"https://www.youtube.com/watch?v=RJ8PPwv-tEE"
driver = getWebDriver('firefox', target_url='')

# Switch to first window_handle, open target_url and 
#    then recursively: 
#    1. open new tabs, 
#    2. switch to that tab's window_handle, 
#    3. open target_url
driver.switch_to.window(driver.window_handles[0])
driver.get(target_url)
for i in tqdm_notebook(range(1,10), desc='Mozilla Loop'):
    # open new tab
    driver.execute_script('''window.open("about:blank", "_blank");''')
    # switch to new tab
    driver.switch_to.window(driver.window_handles[i])
    driver.get(target_url)

C:\Users\raysu\Anaconda3\selenium\webdriver\firefox\geckodriver.exe


A Jupyter Widget

### Check if the following opens a new tab

When tested on a Windows 10 machine, the following code did not create any new tab.

In [87]:
try:
    print('Open Tabs (Before): {}'.format(len(driver.window_handles)))
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('T').key_up(Keys.CONTROL).perform()
    print('Open Tabs (After): {}'.format(len(driver.window_handles)))
except:
    pass

### Close All Tabs One by One

In [86]:
print('Open Tabs: {}'.format(len(driver.window_handles)))
for i, window_handle in tqdm_notebook(enumerate(driver.window_handles), desc='Close Tabs'):
    driver.switch_to.window(window_handle)
    driver.close()
    try:
        open_tabs_left = len(driver.window_handles)
    except:
        open_tabs_left = 0
    print('{}. Open Tabs: {} | Closed: {}'.format(i, open_tabs_left, window_handle))
    time.sleep(0.5)

Open Tabs: 10


A Jupyter Widget

0. Open Tabs: 9 | Closed: 6442450945
1. Open Tabs: 8 | Closed: 6442450953
2. Open Tabs: 7 | Closed: 6442450957
3. Open Tabs: 6 | Closed: 6442450961
4. Open Tabs: 5 | Closed: 6442450965
5. Open Tabs: 4 | Closed: 6442450969
6. Open Tabs: 3 | Closed: 6442450973
7. Open Tabs: 2 | Closed: 6442450977
8. Open Tabs: 1 | Closed: 6442450981
9. Open Tabs: 0 | Closed: 6442450985

